In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
# importovanie potrebných knižníc

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/665045608.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# inštalácia balíkov python
%pip install mlxtend
%pip install - -upgrade pip
%pip install networkx

Note: you may need to restart the kernel to use updated packages.

Usage:   
  /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install [options] <requirement specifier> [package-index-options] ...
  /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install [options] -r <requirements file> [package-index-options] ...
  /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install [options] [-e] <vcs project url> ...
  /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install [options] [-e] <local project path> ...
  /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# inicializácia knižníc pre asociačné pravidlá
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [4]:
#načítanie datasetu
df = pd.read_csv("stroke_data.csv")
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [5]:
# odstránenie stĺpca "id"
df = df.drop(columns=['id'])

In [6]:
# premenovanie atribútov z angličtiny do slovenčiny
df = df.rename(columns={'age': 'vek'})
df = df.rename(columns={'heart_disease': 'srdcova_choroba'})
df = df.rename(columns={'work_type': 'typ_prace'})
df = df.rename(columns={'smoking_status': 'fajcenie'})
df = df.rename(columns={'stroke': 'mrtvica'})
df = df.rename(columns={'Residence_type': 'typ_rezidencie'})
df = df.rename(columns={'avg_glucose_level': 'priemerna_hladina_glukozy'})
df = df.rename(columns={'gender': 'pohlavie'})
df = df.rename(columns={'ever_married': 'manželstvo'})
df = df.rename(columns={'hypertension': 'krvny_tlak'})

In [7]:
# zobrazenie prvých 5 riadkov datasetu
df.head(5)

,pohlavie,vek,krvny_tlak,srdcova_choroba,manželstvo,typ_prace,typ_rezidencie,priemerna_hladina_glukozy,bmi,fajcenie,mrtvica
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [8]:
# spočítanie jednotlivých hodnôt atribútu mrtvica
df.mrtvica.value_counts()

mrtvica
0    4861
1     249
Name: count, dtype: int64

In [9]:
# nahradenie chýbajúcich hodnôt v stĺpci "bmi" priemerom všetkých hodnôt v tomto stĺpci
df['bmi'].fillna(df['bmi'].mean(), inplace=True)

/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/3647075269.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bmi'].fillna(df['bmi'].mean(), inplace=True)


In [10]:
# prevedenie textových hodnôt na číselné
df["pohlavie"] = df["pohlavie"].replace({"Male":0, "Female":1})
df["manželstvo"] = df["manželstvo"].replace({"No":0, "Yes":1})
df["fajcenie"] = df["fajcenie"].replace({"Unknown":0, "smokes":1, "formerly smoked":2, "never smoked":3 })
df["typ_prace"] = df["typ_prace"].replace({"Private":0, "Self-employed":1, "Govt_job":2})
df["typ_rezidencie"] = df["typ_rezidencie"].replace({"Urban":0, "Rural":1})

/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/1893360737.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["manželstvo"] = df["manželstvo"].replace({"No":0, "Yes":1})
/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/1893360737.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["fajcenie"] = df["fajcenie"].replace({"Unknown":0, "smokes":1, "formerly smoked":2, "never smoked":3 })
/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/1893360737.py:6: FutureWarning: Downcasting b

In [11]:
# zobrazenie informácií o atribútoch v datasete
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pohlavie                   5110 non-null   object 
 1   vek                        5110 non-null   float64
 2   krvny_tlak                 5110 non-null   int64  
 3   srdcova_choroba            5110 non-null   int64  
 4   manželstvo                 5110 non-null   int64  
 5   typ_prace                  5110 non-null   object 
 6   typ_rezidencie             5110 non-null   int64  
 7   priemerna_hladina_glukozy  5110 non-null   float64
 8   bmi                        5110 non-null   float64
 9   fajcenie                   5110 non-null   int64  
 10  mrtvica                    5110 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 439.3+ KB


In [12]:
# prevod kategorických na číselné hodnoty
df['pohlavie'], _ = pd.factorize(df['pohlavie'])
df['typ_prace'], _ = pd.factorize(df['typ_prace'])

In [13]:
# prevedenie hodnôt na celé čísla
df['vek'] = df['vek'].astype('int64')
df['priemerna_hladina_glukozy'] = df['priemerna_hladina_glukozy'].astype('int64')
df['bmi'] = df['bmi'].astype('int64')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   pohlavie                   5110 non-null   int64
 1   vek                        5110 non-null   int64
 2   krvny_tlak                 5110 non-null   int64
 3   srdcova_choroba            5110 non-null   int64
 4   manželstvo                 5110 non-null   int64
 5   typ_prace                  5110 non-null   int64
 6   typ_rezidencie             5110 non-null   int64
 7   priemerna_hladina_glukozy  5110 non-null   int64
 8   bmi                        5110 non-null   int64
 9   fajcenie                   5110 non-null   int64
 10  mrtvica                    5110 non-null   int64
dtypes: int64(11)
memory usage: 439.3 KB


In [15]:
# počet prazdných hodnôt v jednotlivých atribútoch
df.isna().sum()

pohlavie                     0
vek                          0
krvny_tlak                   0
srdcova_choroba              0
manželstvo                   0
typ_prace                    0
typ_rezidencie               0
priemerna_hladina_glukozy    0
bmi                          0
fajcenie                     0
mrtvica                      0
dtype: int64

In [16]:
# všetky atribúty datasetu
cols = ["pohlavie", "vek", "krvny_tlak", "srdcova_choroba", "manželstvo", "priemerna_hladina_glukozy", "fajcenie", "typ_prace", "typ_rezidencie", "bmi"]

In [17]:
# normalizácia
scaler = MinMaxScaler()
df[cols] = scaler.fit_transform(df[cols])
df[cols].head()

,pohlavie,vek,krvny_tlak,srdcova_choroba,manželstvo,priemerna_hladina_glukozy,fajcenie,typ_prace,typ_rezidencie,bmi
0,0.0,0.817073,0.0,1.0,1.0,0.800926,0.666667,0.00,0.0,0.298851
1,0.5,0.743902,0.0,0.0,1.0,0.680556,1.000000,0.25,1.0,0.206897
2,0.0,0.975610,0.0,1.0,1.0,0.231481,1.000000,0.00,1.0,0.252874
3,0.5,0.597561,0.0,0.0,1.0,0.537037,0.333333,0.00,0.0,0.275862
4,0.5,0.963415,1.0,0.0,1.0,0.550926,1.000000,0.25,1.0,0.160920


In [18]:
# rozdelenie dát na trénovaciu a testovaciu časť (70:30)
x = df[["pohlavie", "vek", "krvny_tlak", "srdcova_choroba", "manželstvo", "priemerna_hladina_glukozy", "fajcenie", "typ_prace", "typ_rezidencie", "bmi"]]
y = df["mrtvica"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=1)

In [19]:
# Prevod trénovacích dát na binárne hodnoty
x_train_binary = x_train.applymap(lambda x: 1 if x else 0)

# Generovanie častých množín
frequent_itemsets = apriori(x_train_binary, min_support=0.01, use_colnames=True)

# Generovanie asociačných pravidiel
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

# Výpis prvých 10 častých množín
print("----------------------------------------")
print(frequent_itemsets.head(10))
print("----------------------------------------")

/var/folders/lc/0tq3t3m97j18yp2pgzc9h2vw0000gn/T/ipykernel_97787/1606765256.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x_train_binary = x_train.applymap(lambda x: 1 if x else 0)
/opt/homebrew/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


----------------------------------------
    support                     itemsets
0  0.579536                   (pohlavie)
1  0.992172                        (vek)
2  0.098406                 (krvny_tlak)
3  0.053676            (srdcova_choroba)
4  0.659212                 (manželstvo)
5  0.993570  (priemerna_hladina_glukozy)
6  0.696673                   (fajcenie)
7  0.427453                  (typ_prace)
8  0.497065             (typ_rezidencie)
9  0.999720                        (bmi)
----------------------------------------
